In [2]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SequentialEvaluator
from sentence_transformers.readers import STSBenchmarkDataReader
import logging
import sys
import os
import torch
import numpy as np
import zipfile
import io
import pandas as pd

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [31]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v2'
# Load a named sentence model (based on BERT). This will download the model from our server.
# Alternatively, you can also pass a filepath to SentenceTransformer()
model = SentenceTransformer(model_name)

2023-12-21 10:56:31 - Load pretrained SentenceTransformer: sentence-transformers/distiluse-base-multilingual-cased-v2
2023-12-21 10:56:33 - Use pytorch device: cuda


In [3]:
logging.info("Read STS test dataset")
filepath = '../stsbenchmark/sts-test_th.csv'
thai_sts = pd.read_csv(filepath,header=None).dropna()
thai_sts.columns = ['text_1','text_2','correlation']

In [4]:
thai_sts

,text_1,text_2,correlation
0,ผู้หญิงกำลังจัดแต่งทรงผมของเธอ,ผู้หญิงคนหนึ่งกำลังแปรงผมของเธอ,2.5
1,กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด,กลุ่มเด็กผู้ชายกำลังเล่นฟุตบอลบนชายหาด,3.6
2,ผู้หญิงคนหนึ่งกำลังวัดข้อเท้าของผู้หญิงคนอื่น,ผู้หญิงวัดข้อเท้าของผู้หญิงคนอื่น,5.0
3,ผู้ชายกำลังตัดแตงกวา,ผู้ชายกำลังหั่นแตงกวา,4.2
4,ผู้ชายกำลังเล่นพิณ,ผู้ชายกำลังเล่นแป้นพิมพ์,1.5
...,...,...,...
1374,ฟิลิปปินส์แคนาดาให้คำมั่นว่าจะเพิ่มความสัมพันธ...,ฟิลิปปินส์ช่วยประหยัด 100 หลังจมูก,0.0
1375,ชาวอิสราเอลบาร์ชาวปาเลสไตน์จากเมืองเก่าของเยรู...,การแก้ปัญหาสองรัฐระหว่างชาวปาเลสไตน์อิสราเอลพา...,1.0
1376,คุณรู้เกี่ยวกับหน่วยสืบราชการลับมากแค่ไหน?,ผู้ร่างกฎหมายจากทั้งสองฝ่ายแสดงความไม่พอใจที่ห...,1.0
1377,โอบามาดิ้นรนเพื่อบรรเทาความกลัวของซาอุดิอาระเบ...,พม่าพยายามดิ้นรนเพื่อสรุปรายชื่อผู้มีสิทธิเลือ...,0.0


In [32]:
test_evaluator = EmbeddingSimilarityEvaluator(thai_sts['text_1'], thai_sts['text_2'], thai_sts['correlation'], batch_size=16, show_progress_bar=False)
test_evaluator(model) #ใช้แค่ cosine-similarity: spearman: XXXXX

2023-12-21 10:56:35 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
2023-12-21 10:56:37 - Cosine-Similarity :	Pearson: 0.6326	Spearman: 0.6537
2023-12-21 10:56:37 - Manhattan-Distance:	Pearson: 0.6351	Spearman: 0.6353
2023-12-21 10:56:37 - Euclidean-Distance:	Pearson: 0.6468	Spearman: 0.6494
2023-12-21 10:56:37 - Dot-Product-Similarity:	Pearson: 0.5597	Spearman: 0.5602


0.6536675084906219

# Cohere embedding

In [5]:
import cohere
co = cohere.Client('YOUR COHERE API KEY')

In [8]:
from sklearn.metrics.pairwise import paired_cosine_distances
from scipy.stats import pearsonr, spearmanr
import numpy as np

In [21]:
text1 = thai_sts['text_1'].values.tolist()
text2 = thai_sts['text_2'].values.tolist()
label = thai_sts['correlation'].values.tolist()

In [25]:
bs = 96
embed1 = []
embed2 = []
for i in range(len(text1)//bs+1):
    embed1.append(co.embed(
      texts=thai_sts['text_1'][(i*bs):((i+1)*bs)].values.tolist(),
      model='embed-multilingual-v2.0',
    ).embeddings)
    embed2.append(co.embed(
      texts=thai_sts['text_2'][(i*bs):((i+1)*bs)].values.tolist(),
      model='embed-multilingual-v2.0',
    ).embeddings)

In [29]:
embed1_final = np.concatenate(embed1,0)
embed2_final = np.concatenate(embed2,0)
embed1_final.shape, embed2_final.shape

((1379, 768), (1379, 768))

In [30]:
cosine_scores = 1 - (paired_cosine_distances(embed1_final, embed2_final))
eval_spearman_cosine, _ = spearmanr(label, cosine_scores)
eval_spearman_cosine

0.6802529676942305

# BGE-M3

In [ ]:
from sklearn.metrics.pairwise import paired_cosine_distances
from scipy.stats import pearsonr, spearmanr
import numpy as np

In [4]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True) 

Fetching 23 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 140420.66it/s]


In [5]:
text1 = thai_sts['text_1'].values.tolist()
text2 = thai_sts['text_2'].values.tolist()
label = thai_sts['correlation'].values.tolist()

In [10]:
embed1 = model.encode(text1)['dense_vecs']
embed2 = model.encode(text2)['dense_vecs']

Inference Embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 115/115 [00:03<00:00, 37.35it/s]


In [9]:
cosine_scores = 1 - (paired_cosine_distances(embed1, embed2))
eval_spearman_cosine, _ = spearmanr(label, cosine_scores)
eval_spearman_cosine

0.7722125470883906